# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: 2-3 Tree

In [2]:
import time

In [1]:
!pip install --upgrade bokeh==2.4.3

In [3]:
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
from time import sleep

# Algorithm

In [4]:
%%time
class Node:
    def __init__(self, value, left=None, right=None, red=False):
        self.value = value
        self.left = left
        self.right = right
        self.red = red

    @staticmethod
    def red(node):
        return node and node.red

    def rotate_left(self):
        if not Node.red(self.left) and Node.red(self.right):
            child = self.right
            self.right, child.left = child.left, self
            self.red, child.red = True, self.red
            return child
        else:
            return self

    def rotate_right(self):
        if Node.red(self.left) and Node.red(self.left.left):
            child = self.left
            self.left, child.right = child.right, self
            self.red, child.red = True, self.red
            return child
        else:
            return self

    def flip_colors(self):
        if Node.red(self.left) and Node.red(self.right):
            self.red, self.left.red, self.right.red = True, False, False

CPU times: user 41 µs, sys: 7 µs, total: 48 µs
Wall time: 52 µs


In [5]:
%%time
def insert(node, value, root=True):
    if not node:
        return Node(value, red=not root)

    # insert value
    if value == node.value:
        pass
    elif value < node.value:
        node.left = insert(node.left, value, root=False)
    else:
        node.right = insert(node.right, value, root=False)

    # update tree w.r.t invariants
    node = node.rotate_left()
    node = node.rotate_right()
    node.flip_colors()

    # keep root black
    if root:
        node.red = False

    return node

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 9.3 µs


In [6]:
%%time
def draw(node, red, black, xlo=0, xhi=1, x=.5, y=0):
    if node:
        x_, y_ = (xlo + xhi) / 2, y - 1
        (red if node.red else black).append([x, y, x_, y_])
        draw(node.left, red, black, xlo, x_, x_, y_)
        draw(node.right, red, black, x_, xhi, x_, y_)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.54 µs


# Run

In [7]:
%%time
# insert few values
root = None
for i in range(5):
    root = insert(root, i)

CPU times: user 43 µs, sys: 7 µs, total: 50 µs
Wall time: 67.7 µs


In [8]:
%%time
output_notebook()

# get plot data
red, black = [], []
draw(root, red, black)

# plot
plot = figure(plot_height=400, plot_width=800)
plot.axis.visible = False
plot.grid.visible = False

red_segment = plot.segment(*zip(*red), color='#ff0000')
black_segment = plot.segment(*zip(*black), color='#000000')

handle = show(plot, notebook_handle=True)

CPU times: user 99 ms, sys: 2.82 ms, total: 102 ms
Wall time: 132 ms


In [9]:
%%time
for _ in range(100):
    for i in np.random.randint(0, 1000, 5):
        root = insert(root, i)

    red, black = [], []
    draw(root, red, black)

    x0, y0, x1, y1 = red and zip(*red) or [tuple()] * 4
    red_segment.data_source.data.update({
        'x0': x0, 'y0': y0, 'x1': x1, 'y1': y1
    })
    x0, y0, x1, y1 = black and zip(*black) or [tuple()] * 4
    black_segment.data_source.data.update({
        'x0': x0, 'y0': y0, 'x1': x1, 'y1': y1
    })

    push_notebook()
    sleep(.1)

CPU times: user 691 ms, sys: 35.4 ms, total: 726 ms
Wall time: 11.1 s


# The End